For CPAIOR 2023

In [51]:
import pandas as pd
import numpy as np
from statistics import geometric_mean

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [52]:
# import result sets
df_ml = pd.read_csv("Set_1_analysis.csv")
df_ml = df_ml.drop(['GAP W/ CUTS', 'GAP W/O CUTS'], axis = 1)
df_ml.rename(columns={'INITIAL LP':'INITIAL LP W/O CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/O CUTS'}, inplace=True)
df = pd.read_csv("results_1_new.csv")
analysis_ml = df_ml.merge(df, on = ['NAME', 'RANDOM SEED'], how = 'left')
analysis_ml = analysis_ml.drop(['BEST SOLUTION', 'BEST PRIMAL BOUND', 'GAP', 'No. OF CUTS GENERATED', 'No. OF CUTS APPLIED',
                               'No. OF CUTS PRESENT IN LP AT THE END', 'No. OF NODES', 'RANDOM SEED', 'CUT SETTING',
                               'STATUS', 'END', 'SOLUTION TIME'], axis = 1)
analysis_ml.rename(columns={'INITIAL LP':'INITIAL LP W/ CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/ CUTS'}, inplace=True)
analysis_ml['CUTS OFF_FIRST ROUND'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME FIRST ROUND']
analysis_ml['CUTS OFF_ROOT END'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME ROOT END']
analysis_ml = analysis_ml.drop(['INITIAL LP TIME', 'TIME W/O CUTS', 'CUTS OFF_FIRST ROUND'], axis = 1)

analysis_ml = analysis_ml[['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 'PRESOLVING TIME W/ CUTS', 'PRESOLVING TIME W/O CUTS',
                            'TIME FIRST ROUND', 'TIME ROOT END','NODES W/ CUTS', 'NODES W/O CUTS', 'INITIAL LP W/ CUTS', 
                           'INITIAL LP W/O CUTS', 'FIRST ROUND CUT',
                           'ROUND OF CUTS', 'OBJECTIVE VALUE ROOT NODE', 'vars', 'bin_vars', 'int_vars', 'cont_vars', 
                           'constr', 'linsetpart_constr', 'linsetpack_constr', 'linsetcov_constr', 'lincard_constr', 
                           'lineqknap_constr', 'linbinpack_constr', 'linknaps_constr', 'linintknap_constr', 'linmixbin_constr',
                           'nonzeroes', 'obj_dyn', 'dynamism_mean', 'presol_vars','presol_int_vars', 'presol_constr']]

In [53]:
pd.set_option('display.max_columns', None)

In [54]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,TIME ROOT END,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,11.088517,15.713885,0.067284,0.062201,1.426593,3.501355,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
1,10teams.mps.gz,11.186637,15.838753,0.057992,0.060087,1.378128,3.394305,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
2,10teams.mps.gz,11.044666,15.637111,0.055842,0.058780,1.350495,3.305622,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
3,10teams.mps.gz,10.835616,15.641646,0.052585,0.059116,1.293845,3.174460,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
4,10teams.mps.gz,10.833424,15.666084,0.053544,0.058801,1.301532,3.188138,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,wachplan.mps.gz,1427.930347,1900.337469,1.165874,1.552340,2.689512,98.732839,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1771,wachplan.mps.gz,1444.662760,1901.090928,1.031269,1.914422,2.536143,96.248496,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1772,wachplan.mps.gz,1431.659624,1833.795217,1.088057,1.507830,2.595248,96.643203,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1773,wachplan.mps.gz,1433.459125,1856.035662,1.125112,1.547767,2.649715,97.447706,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025


In [55]:
analysis_ml = analysis_ml.groupby('NAME').mean()
analysis_ml = analysis_ml.reset_index()

In [56]:
time_label_1 = []

for index, row in analysis_ml.iterrows():
    if row['TIME W/ CUTS'] > row['CUTS OFF_ROOT END']:
        time_label_1.append(-1)
    elif row['TIME W/ CUTS'] <= row['CUTS OFF_ROOT END']:
        time_label_1.append(1)
    
analysis_ml['Time Label'] = time_label_1

In [57]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,TIME ROOT END,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,10teams.mps.gz,10.997772,15.699496,0.057449,0.059797,1.350119,3.312776,30,276,2.000000e+01,2.000000e+01,9.170000e+02,13,9.240000e+02,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108,1
1,22433.mps.gz,2.535930,3.478681,0.046082,0.025466,0.172883,2.828100,1,5,2.124053e+04,2.124053e+04,2.127800e+04,58,2.147114e+04,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267,1
2,23588.mps.gz,5.527370,4.372027,0.027812,0.031946,0.140688,1.972278,576,981,7.649866e+03,7.649866e+03,7.660000e+03,34,7.765000e+03,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981,-1
3,30_70_45_05_100.mps.gz,282.653795,402.324263,1.986556,1.625184,18.567687,162.244480,46,201,-5.336000e+03,-5.336000e+03,8.500000e+00,13,9.000000e+00,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654,1
4,30_70_45_095_100.mps.gz,743.997969,246.544417,1.994498,2.143665,19.518728,190.829956,201,1,-6.132000e+03,-6.132000e+03,3.000000e+00,12,3.000000e+00,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,triptim1.mps.gz,2446.382658,1200.984877,6.005649,5.952476,226.483793,510.293679,78,11,-2.034000e+00,-2.034000e+00,2.286809e+01,20,2.286809e+01,10.310784,0.680619,0.319148,0.000233,9.661798,0.000000,0.130460,0.257099,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001092,0.00000,0.002040,10.153818,0.372767,9.661352,-1
351,umts.mps.gz,1079.840992,11640.938871,0.179507,0.230250,0.892776,36.086503,144227,3106370,1.460426e+06,1.460426e+06,2.912957e+07,30,2.914217e+07,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152,1
352,unitcal_7.mps.gz,940.799926,10890.250070,6.937303,6.065447,64.953906,326.517054,705,196989,1.028415e+06,1.028415e+06,1.946934e+07,86,1.960908e+07,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349,1
353,var-smallemery-m6j6.mps.gz,7171.038360,10328.248584,8.873553,8.342061,79.559820,82.023799,685889,836578,-1.697219e+03,-1.697219e+03,-1.549250e+02,1,-1.549250e+02,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321,1


In [58]:
X = analysis_ml.iloc[0:, 0:-1]
y = analysis_ml.iloc[0:, -1]

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                       'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
cols = X_train.copy()
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
cols

(284, 29) (284,)
(71, 29) (71,)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,TIME ROOT END,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
298,0.025660,0.042668,0.084510,4.266467,-9.200187e+03,-9.200187e+03,-9.316389e+02,14,-8.325706e+02,6.733402,0.057143,0.000000,0.942857,7.083388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.114094,0.003428,0.827942,0.234359,6.733402,0.000000,7.083388
247,1.227221,1.474001,41.522818,308.243039,-2.492000e+03,-2.492000e+03,2.760000e+02,12,2.760000e+02,10.386438,0.883395,0.000000,0.116605,9.522081,0.000000,0.392224,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.237443,0.000423,3.000000,0.000000,9.092345,0.000000,9.239025
309,0.029698,0.026762,0.048170,0.123641,1.324444e+01,1.324444e+01,1.400000e+01,13,1.400000e+01,6.133398,0.976139,0.000000,0.023861,4.219508,0.000000,0.000000,0.000000,0.147059,0.0,0.000000,0.000000,0.000000,0.661765,0.026668,0.000000,0.758467,5.888878,0.000000,3.970292
336,0.198174,0.307568,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,8.058960,1.000000,0.000000,0.000000,9.158205,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.000000,0.000000,0.244627,0.001267,0.000000,0.213065,7.667626,0.000000,8.767173
142,12.828483,19.519242,35.412962,60.565992,-2.599800e+04,-2.599800e+04,-2.287477e+04,34,-2.192093e+04,9.234057,0.975488,0.024512,0.000000,9.158942,0.112959,0.605853,0.002948,0.232446,0.0,0.000000,0.000105,0.000000,0.000105,0.001384,2.397940,0.054545,9.234057,0.019336,9.158942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2.558282,1.902435,12.338459,445.718916,4.442776e+08,4.442776e+08,4.444208e+08,27,4.449067e+08,9.295967,1.000000,0.000000,0.000000,6.715383,0.000000,0.756364,0.020606,0.000000,0.0,0.000000,0.000000,0.000000,0.219394,0.035195,1.531250,0.098539,9.295967,0.000000,6.715383
192,11.176662,15.067371,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,10.619814,0.127656,0.000000,0.872344,11.617520,0.002072,0.000486,0.000000,0.000000,0.0,0.000108,0.000054,0.000000,0.350567,0.000121,2.595280,0.018692,10.619814,0.000000,11.617520
117,0.880457,1.154996,1.493279,4.440735,3.000000e+00,3.000000e+00,3.000000e+00,20,4.000000e+00,8.974745,0.970130,0.029870,0.000000,9.558388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.094330,0.000000,0.000737,0.000000,2.724850,8.023225,0.037365,8.583917
47,0.247610,0.309220,6.985466,18.496850,3.000000e+00,3.000000e+00,3.000000e+00,11,3.000000e+00,9.128696,1.000000,0.000000,0.000000,9.041567,0.272641,0.727359,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000434,0.000000,0.000000,9.128696,0.000000,9.041567


In [60]:
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [61]:
cols = cols.iloc[0:, 0:]

bestfeatures = SelectKBest(score_func=f_classif, k = 'all')
fit = bestfeatures.fit(X_train,y_train)
scores = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(cols.columns)
featureScores = pd.concat([columns,scores],axis=1)
featureScores
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(29,'Score')) #print best features
data=featureScores.nlargest(29,'Score')

                     Features     Score
22           linmixbin_constr  4.624454
16           linsetcov_constr  4.402348
23                  nonzeroes  3.423498
13                     constr  3.397892
8   OBJECTIVE VALUE ROOT NODE  3.261068
4          INITIAL LP W/ CUTS  2.861642
5         INITIAL LP W/O CUTS  2.861642
28              presol_constr  2.755184
7               ROUND OF CUTS  2.621683
15          linsetpack_constr  1.610482
11                   int_vars  1.223737
17             lincard_constr  1.179362
19          linbinpack_constr  1.061472
21          linintknap_constr  0.913930
25              dynamism_mean  0.491767
18           lineqknap_constr  0.487896
6             FIRST ROUND CUT  0.440752
14          linsetpart_constr  0.384992
27            presol_int_vars  0.368633
9                        vars  0.289634
3               TIME ROOT END  0.209276
26                presol_vars  0.197017
1    PRESOLVING TIME W/O CUTS  0.160531
10                   bin_vars  0.108356


In [62]:
X = analysis_ml.drop(['obj_dyn', 'PRESOLVING TIME W/ CUTS', 'cont_vars', 'linknaps_constr',
                      'TIME FIRST ROUND', 'bin_vars', 'PRESOLVING TIME W/O CUTS',
                      'Time Label'], axis = 1)
y = analysis_ml.iloc[0:, -1]

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

test_df = X_test.copy()
train_df = X_train.copy()

X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 'NODES W/ CUTS', 
                        'NODES W/O CUTS'], axis = 1)
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END',
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(284, 22) (284,)
(71, 22) (71,)


In [64]:
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [65]:
model_params={'svm':{'model': SVC(kernel = 'rbf', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 'gamma': [1e-2, 1e-3, 1e-4]
                             }
                    },
              'rfm':{'model': RandomForestClassifier(random_state = 0), 
                    'params':{'n_estimators':[100, 200, 500], 
                             'max_depth':[5, 10, 15]
                             }
                    },
              'lrm':{'model': LogisticRegression(solver='lbfgs', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 
                              'max_iter':[100, 500, 1000]
                             }
                    },
              'et':{'model': ExtraTreesClassifier(random_state=0), 
                    'params':{'n_estimators': [100, 200, 500], 
                              'max_depth': [5, 10, 15]
                             }
                    },
              'nn':{'model': MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', 
                                           power_t=0.5, max_iter=500, shuffle=True, tol=0.00001, verbose=True, 
                                           early_stopping=False, validation_fraction=0.1, random_state=0),
                    'params':{'hidden_layer_sizes': [(5, ), (10, ), (15, )],
                                 'learning_rate_init': [1e-1, 1e-2, 1e-3],
                                 'alpha':  [1e-2, 1e-3, 1e-4],
                                 'activation': ['tanh', 'relu', 'logistic']
                             }
                    }
             }
scores=[]
for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], cv = None, return_train_score = False, n_jobs = -1)
    clf.fit(X_train, y_train)
    scores.append({'model': model_name, 'best_score':clf.best_score_, 'best_params':clf.best_params_})

C:\Users\ambar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [97]:
scores=pd.DataFrame(scores).sort_values(by = ['best_score'], ascending = False)
scores.to_csv('set_1_scores_new.csv')

KeyError: 'best_score'

In [ ]:
scores = OrderedDict()
target_names = ['-1', '1']

print('SUPPORT VECTOR MACHINES')
clf = SVC(random_state = 0, C = 100, gamma = 0.001, kernel = 'rbf')
clf.fit(X_train, y_train)
y_train_pred_svm = clf.predict(X_train)
y_test_pred_svm = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_svm)
mse_test = mean_squared_error(y_test, y_test_pred_svm)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_svm, target_names=target_names))
scores['SVC'] = [score, metrics.precision_score(y_test, y_test_pred_svm), metrics.recall_score(y_test, y_test_pred_svm),
                      metrics.f1_score(y_test, y_test_pred_svm), mse_test, mse_train]

print('##################################')
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_nn)
mse_test = mean_squared_error(y_test, y_test_pred_nn)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_nn, target_names=target_names))
scores['NN'] = [score, metrics.precision_score(y_test, y_test_pred_nn), metrics.recall_score(y_test, y_test_pred_nn),
                      metrics.f1_score(y_test, y_test_pred_nn), mse_test, mse_train]

print('##################################')
print('DECISION TREES')
clf = ExtraTreesClassifier(random_state=0, max_depth = 5, n_estimators = 200)
clf.fit(X_train, y_train)
y_train_pred_et = clf.predict(X_train)
y_test_pred_et = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_et)
mse_test = mean_squared_error(y_test, y_test_pred_et)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_et, target_names=target_names))
scores['ET'] = [score, metrics.precision_score(y_test, y_test_pred_et), 
                         metrics.recall_score(y_test, y_test_pred_et), 
                         metrics.f1_score(y_test, y_test_pred_et), mse_test, mse_train]

print('##################################')
print('LOGISTIC REGRESSION')
clf = LogisticRegression(solver='lbfgs', random_state = 0, C = 100, max_iter = 100)
clf.fit(X_train, y_train)
y_train_pred_lr = clf.predict(X_train)
y_test_pred_lr = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_lr)
mse_test = mean_squared_error(y_test, y_test_pred_lr)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_lr, target_names=target_names))
scores['LR'] = [score, metrics.precision_score(y_test, y_test_pred_lr), metrics.recall_score(y_test, y_test_pred_lr),
                      metrics.f1_score(y_test, y_test_pred_lr), mse_test, mse_train]

print('##################################')
print('RANDOM FOREST')
clf = RandomForestClassifier(random_state = 0, max_depth = 5, n_estimators = 100)
clf.fit(X_train, y_train)
y_train_pred_rf = clf.predict(X_train)
y_test_pred_rf = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_rf)
mse_test = mean_squared_error(y_test, y_test_pred_rf)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_rf, target_names=target_names))
scores['RF'] = [score, metrics.precision_score(y_test, y_test_pred_rf), metrics.recall_score(y_test, y_test_pred_rf),
                      metrics.f1_score(y_test, y_test_pred_rf), mse_test, mse_train]

In [98]:
scores_df = pd.DataFrame.from_dict(scores, orient='index')
scores_df.columns=['accuracy', 'precision', 'recall', 'f1-score', 'mse test', 'mse train']
scores_df

,accuracy,precision,recall,f1-score,mse test,mse train
SVC,0.605634,0.621212,0.931818,0.745455,1.577465,1.098592
NN,0.549296,0.625000,0.681818,0.652174,1.802817,0.422535
ET,0.633803,0.628571,1.000000,0.771930,1.464789,1.084507
LR,0.661972,0.651515,0.977273,0.781818,1.352113,1.056338
RF,0.633803,0.628571,1.000000,0.771930,1.464789,0.690141


In [99]:
test_df['PLabel NN'] = y_test_pred_nn
test_df['PLabel ET'] = y_test_pred_et
test_df['PLabel RF'] = y_test_pred_rf

test_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = test_df.merge(test_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label 3'])
analysis['Minimum Time'] = analysis[['CUTS OFF_ROOT END', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Minimum Time,Minimum Nodes
0,30n20b8.mps.gz,152.430964,279.361391,1617,13632,1,1,1,152.430964,1617
1,mspsphard01i.mps.gz,10493.451548,3655.648885,323371,152457,1,1,1,3655.648885,152457
2,rail507.mps.gz,183.736345,234.877657,836,1010,1,1,1,183.736345,836
3,neos-5193246-nerang.mps.gz,1171.813190,1700.243612,7082,17117,-1,1,1,1171.813190,7082
4,gen-ip002.mps.gz,1482.181497,1182.264337,5859548,5112787,1,1,1,1182.264337,5112787
...,...,...,...,...,...,...,...,...,...,...
66,neos-3611689-kaihu.mps.gz,1.927490,20.897175,21,29199,1,1,1,1.927490,21
67,neos-1601936.mps.gz,11527.267046,19466.974637,14529,1,-1,1,1,11527.267046,1
68,neos-1425699.mps.gz,0.011907,0.031720,1,1,1,1,1,0.011907,1
69,neos-4322846-ryton.mps.gz,818.066564,905.172764,664,522,-1,1,1,818.066564,522


In [100]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == -1.0:
        rf_pred_time.append(row['CUTS OFF_ROOT END'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == -1.0:
        nn_pred_time.append(row['CUTS OFF_ROOT END'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == -1.0:
        et_pred_time.append(row['CUTS OFF_ROOT END'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,30n20b8.mps.gz,152.430964,279.361391,1617,13632,1,1,1,152.430964,1617,152.430964,152.430964,152.430964,1617,1617,1617
1,mspsphard01i.mps.gz,10493.451548,3655.648885,323371,152457,1,1,1,3655.648885,152457,10493.451548,10493.451548,10493.451548,323371,323371,323371
2,rail507.mps.gz,183.736345,234.877657,836,1010,1,1,1,183.736345,836,183.736345,183.736345,183.736345,836,836,836
3,neos-5193246-nerang.mps.gz,1171.813190,1700.243612,7082,17117,-1,1,1,1171.813190,7082,1171.813190,1700.243612,1171.813190,7082,17117,7082
4,gen-ip002.mps.gz,1482.181497,1182.264337,5859548,5112787,1,1,1,1182.264337,5112787,1482.181497,1482.181497,1482.181497,5859548,5859548,5859548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,neos-3611689-kaihu.mps.gz,1.927490,20.897175,21,29199,1,1,1,1.927490,21,1.927490,1.927490,1.927490,21,21,21
67,neos-1601936.mps.gz,11527.267046,19466.974637,14529,1,-1,1,1,11527.267046,1,11527.267046,19466.974637,11527.267046,14529,1,14529
68,neos-1425699.mps.gz,0.011907,0.031720,1,1,1,1,1,0.011907,1,0.011907,0.011907,0.011907,1,1,1
69,neos-4322846-ryton.mps.gz,818.066564,905.172764,664,522,-1,1,1,818.066564,522,818.066564,905.172764,818.066564,664,522,664


In [101]:
testing = analysis.copy()

In [102]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [103]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [104]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [105]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_ROOT END'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [106]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      274.255724   347.103758   230.996876   
               [10,18000)     526.420781   657.537397   428.412232   
               [100,18000)   1152.792787  1279.713679   898.777222   
               [1000,18000)  4195.720833  4190.303578  2823.228908   
               [3600,18000)  7788.063000  8777.731836  5670.386529   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      271.396255   271.396255   281.806623   
               [10,18000)     519.710113   519.710113   542.399525   
               [100,18000)   1132.962045  1132.962045  1182.880706   
               [1000,18000)  4045.723024  4045.723024  4419.029446   
               [3600,18000)  7788.063000  7788.063000  8413.041390   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      5873.772645   12660.610956   5074.993594   
               [10,18000)    11162.501804   23205.981020   9343.101918   
               [100,18000)   22909.893632   39088.398997  18309.681543   
               [1000,18000)  98636.196048  207875.647224  71152.564753   
               [3600,18000)  91266.680556  167606.034070  58645.036046   

Parameter                                                               \
Setting                                RF            ET             NN   
Metric         Bucket                                                    
Geometric Mean [0,18000)      5856.937165   5856.937165    7024.195948   
               [10,18000)    11122.612636  11122.612636   13392.961220   
               [100,18000)   22797.646416  22797.646416   24997.992772   
               [1000,18000)  97620.959528  97620.959528  115417.415645   
               [3600,18000)  91266.680556  91266.680556   93258.401993   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)           71  
               [10,18000)          53  
               [100,18000)         37  
               [1000,18000)        17  
               [3600,18000)         8

In [107]:
imp_time_rf = []
imp_nodes_rf = []

for i, j in zip(t_rf, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_rf.append(imp)

for i, j in zip(n_rf, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_rf.append(imp)

In [108]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_rf
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_rf
avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      274.255724   347.103758   230.996876   
               [10,18000)     526.420781   657.537397   428.412232   
               [100,18000)   1152.792787  1279.713679   898.777222   
               [1000,18000)  4195.720833  4190.303578  2823.228908   
               [3600,18000)  7788.063000  8777.731836  5670.386529   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      271.396255   271.396255   281.806623   
               [10,18000)     519.710113   519.710113   542.399525   
               [100,18000)   1132.962045  1132.962045  1182.880706   
               [1000,18000)  4045.723024  4045.723024  4419.029446   
               [3600,18000)  7788.063000  7788.063000  8413.041390   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      5873.772645   12660.610956   5074.993594   
               [10,18000)    11162.501804   23205.981020   9343.101918   
               [100,18000)   22909.893632   39088.398997  18309.681543   
               [1000,18000)  98636.196048  207875.647224  71152.564753   
               [3600,18000)  91266.680556  167606.034070  58645.036046   

Parameter                                                               \
Setting                                RF            ET             NN   
Metric         Bucket                                                    
Geometric Mean [0,18000)      5856.937165   5856.937165    7024.195948   
               [10,18000)    11122.612636  11122.612636   13392.961220   
               [100,18000)   22797.646416  22797.646416   24997.992772   
               [1000,18000)  97620.959528  97620.959528  115417.415645   
               [3600,18000)  91266.680556  91266.680556   93258.401993   

Parameter                   Instances Improvement (%)            
Setting                                          Time     Nodes  
Metric         Bucket                                            
Geometric Mean [0,18000)           71       -1.042629 -0.286621  
               [10,18000)          53       -1.274773 -0.357350  
               [100,18000)         37       -1.720235 -0.489951  
               [1000,18000)        17       -3.575019 -1.029274  
               [3600,18000)         8        0.000000  0.000000

In [109]:
train_df

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,TIME ROOT END,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,int_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linintknap_constr,linmixbin_constr,nonzeroes,dynamism_mean,presol_vars,presol_int_vars,presol_constr,PLabel NN,PLabel ET,PLabel RF
298,qiu.mps.gz,68.293240,47.141371,4.266467,10652,9207,-9.200187e+03,-9.200187e+03,-9.316389e+02,14,-8.325706e+02,6.733402,0.000000,7.083388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.114094,0.003428,0.234359,6.733402,0.000000,7.083388,-1,1,1
247,neos-933638.mps.gz,14587.569504,1597.435977,308.243039,983,64,-2.492000e+03,-2.492000e+03,2.760000e+02,12,2.760000e+02,10.386438,0.000000,9.522081,0.000000,0.392224,0.000000,0.000000,0.0,0.000000,0.000000,0.237443,0.000423,0.000000,9.092345,0.000000,9.239025,-1,1,-1
309,rlp1.mps.gz,0.211836,0.339914,0.123641,51,225,1.324444e+01,1.324444e+01,1.400000e+01,13,1.400000e+01,6.133398,0.000000,4.219508,0.000000,0.000000,0.000000,0.147059,0.0,0.000000,0.000000,0.661765,0.026668,0.758467,5.888878,0.000000,3.970292,1,1,1
336,supportcase4.mps.gz,3.806084,113.729586,0.000000,1,160,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,8.058960,0.000000,9.158205,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.000000,0.244627,0.001267,0.213065,7.667626,0.000000,8.767173,1,1,1
142,mzzv11.mps.gz,422.340278,1550.475830,60.565992,1356,8880,-2.599800e+04,-2.599800e+04,-2.287477e+04,34,-2.192093e+04,9.234057,0.024512,9.158942,0.112959,0.605853,0.002948,0.232446,0.0,0.000000,0.000000,0.000105,0.001384,0.054545,9.234057,0.019336,9.158942,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,sp98ic.mps.gz,8603.418640,13704.848994,445.718916,141964,432282,4.442776e+08,4.442776e+08,4.444208e+08,27,4.449067e+08,9.295967,0.000000,6.715383,0.000000,0.756364,0.020606,0.000000,0.0,0.000000,0.000000,0.219394,0.035195,0.098539,9.295967,0.000000,6.715383,1,1,1
192,neos-3755335-nizao.mps.gz,16.354954,16.846205,0.000000,1,1,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,10.619814,0.000000,11.617520,0.002072,0.000486,0.000000,0.000000,0.0,0.000108,0.000000,0.350567,0.000121,0.018692,10.619814,0.000000,11.617520,-1,1,1
117,lectsched-4-obj.mps.gz,5.974980,22.306879,4.440735,5,1395,3.000000e+00,3.000000e+00,3.000000e+00,20,4.000000e+00,8.974745,0.029870,9.558388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.094330,0.000000,0.000737,2.724850,8.023225,0.037365,8.583917,1,1,1
47,chromaticindex128-5.mps.gz,84.350632,1211.104534,18.496850,1316,100314,3.000000e+00,3.000000e+00,3.000000e+00,11,3.000000e+00,9.128696,0.000000,9.041567,0.272641,0.727359,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000434,0.000000,9.128696,0.000000,9.041567,1,1,1


In [110]:
train_df['PLabel NN'] = y_train_pred_nn
train_df['PLabel ET'] = y_train_pred_et
train_df['PLabel RF'] = y_train_pred_rf

train_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = train_df.merge(train_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['CUTS OFF_ROOT END', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,qiu.mps.gz,68.293240,47.141371,10652,9207,-1,1,1,-1,47.141371,9207
1,neos-933638.mps.gz,14587.569504,1597.435977,983,64,-1,1,-1,-1,1597.435977,64
2,rlp1.mps.gz,0.211836,0.339914,51,225,1,1,1,1,0.211836,51
3,supportcase4.mps.gz,3.806084,113.729586,1,160,1,1,1,1,3.806084,1
4,mzzv11.mps.gz,422.340278,1550.475830,1356,8880,1,1,1,1,422.340278,1356
...,...,...,...,...,...,...,...,...,...,...,...
279,sp98ic.mps.gz,8603.418640,13704.848994,141964,432282,1,1,1,1,8603.418640,141964
280,neos-3755335-nizao.mps.gz,16.354954,16.846205,1,1,-1,1,1,1,16.354954,1
281,lectsched-4-obj.mps.gz,5.974980,22.306879,5,1395,1,1,1,1,5.974980,5
282,chromaticindex128-5.mps.gz,84.350632,1211.104534,1316,100314,1,1,1,1,84.350632,1316


In [111]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == -1.0:
        rf_pred_time.append(row['CUTS OFF_ROOT END'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == -1.0:
        nn_pred_time.append(row['CUTS OFF_ROOT END'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == -1.0:
        et_pred_time.append(row['CUTS OFF_ROOT END'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,qiu.mps.gz,68.293240,47.141371,10652,9207,-1,1,1,-1,47.141371,9207,68.293240,47.141371,68.293240,10652,9207,10652
1,neos-933638.mps.gz,14587.569504,1597.435977,983,64,-1,1,-1,-1,1597.435977,64,1597.435977,1597.435977,14587.569504,64,64,983
2,rlp1.mps.gz,0.211836,0.339914,51,225,1,1,1,1,0.211836,51,0.211836,0.211836,0.211836,51,51,51
3,supportcase4.mps.gz,3.806084,113.729586,1,160,1,1,1,1,3.806084,1,3.806084,3.806084,3.806084,1,1,1
4,mzzv11.mps.gz,422.340278,1550.475830,1356,8880,1,1,1,1,422.340278,1356,422.340278,422.340278,422.340278,1356,1356,1356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,sp98ic.mps.gz,8603.418640,13704.848994,141964,432282,1,1,1,1,8603.418640,141964,8603.418640,8603.418640,8603.418640,141964,141964,141964
280,neos-3755335-nizao.mps.gz,16.354954,16.846205,1,1,-1,1,1,1,16.354954,1,16.354954,16.846205,16.354954,1,1,1
281,lectsched-4-obj.mps.gz,5.974980,22.306879,5,1395,1,1,1,1,5.974980,5,5.974980,5.974980,5.974980,5,5,5
282,chromaticindex128-5.mps.gz,84.350632,1211.104534,1316,100314,1,1,1,1,84.350632,1316,84.350632,84.350632,84.350632,1316,1316,1316


In [112]:
training = analysis.copy()

In [113]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [114]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [115]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [116]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_ROOT END'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [117]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      180.289030   277.153358   154.992595   
               [10,18000)     318.077008   483.799437   266.244304   
               [100,18000)    775.129611  1063.621559   613.857516   
               [1000,18000)  3287.332970  4141.305203  2412.842201   
               [3600,18000)  7522.182084  8063.702945  6553.741229   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      168.868150   175.427147   164.576260   
               [10,18000)     294.208609   307.864512   285.542499   
               [100,18000)    699.207706   746.941829   673.246623   
               [1000,18000)  2667.094083  3255.217417  2546.688621   
               [3600,18000)  7229.477495  7468.016843  6806.038164   

Parameter                            Node                              \
Setting                               Cut        No Cut       Perfect   
Metric         Bucket                                                   
Geometric Mean [0,18000)      2610.230290   7071.275598   2308.375949   
               [10,18000)     4381.790018  11259.477754   3788.838502   
               [100,18000)    7925.676084  16008.342301   6614.537456   
               [1000,18000)  28245.745646  52073.861031  21678.679455   
               [3600,18000)  57427.170927  90712.284096  54083.640415   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      2483.432574   2541.630897   2505.401045   
               [10,18000)     4129.005877   4245.242100   4131.618204   
               [100,18000)    7302.769570   7604.709071   7388.242919   
               [1000,18000)  23425.499136  28293.419117  23800.428107   
               [3600,18000)  55460.888756  57290.987755  61215.008557   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)          284  
               [10,18000)         212  
               [100,18000)        133  
               [1000,18000)        40  
               [3600,18000)        10

In [118]:
imp_time_rf = []
imp_nodes_rf = []

for i, j in zip(t_rf, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_rf.append(imp)

for i, j in zip(n_rf, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_rf.append(imp)

In [119]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_rf
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_rf
avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      180.289030   277.153358   154.992595   
               [10,18000)     318.077008   483.799437   266.244304   
               [100,18000)    775.129611  1063.621559   613.857516   
               [1000,18000)  3287.332970  4141.305203  2412.842201   
               [3600,18000)  7522.182084  8063.702945  6553.741229   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      168.868150   175.427147   164.576260   
               [10,18000)     294.208609   307.864512   285.542499   
               [100,18000)    699.207706   746.941829   673.246623   
               [1000,18000)  2667.094083  3255.217417  2546.688621   
               [3600,18000)  7229.477495  7468.016843  6806.038164   

Parameter                            Node                              \
Setting                               Cut        No Cut       Perfect   
Metric         Bucket                                                   
Geometric Mean [0,18000)      2610.230290   7071.275598   2308.375949   
               [10,18000)     4381.790018  11259.477754   3788.838502   
               [100,18000)    7925.676084  16008.342301   6614.537456   
               [1000,18000)  28245.745646  52073.861031  21678.679455   
               [3600,18000)  57427.170927  90712.284096  54083.640415   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      2483.432574   2541.630897   2505.401045   
               [10,18000)     4129.005877   4245.242100   4131.618204   
               [100,18000)    7302.769570   7604.709071   7388.242919   
               [1000,18000)  23425.499136  28293.419117  23800.428107   
               [3600,18000)  55460.888756  57290.987755  61215.008557   

Parameter                   Instances Improvement (%)             
Setting                                          Time      Nodes  
Metric         Bucket                                             
Geometric Mean [0,18000)          284       -6.334761  -4.857721  
               [10,18000)         212       -7.503969  -5.768970  
               [100,18000)        133       -9.794737  -7.859349  
               [1000,18000)        40      -18.867541 -17.065389  
               [3600,18000)        10       -3.891219  -3.423958

In [120]:
df = [training, testing]

t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_ROOT END'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [121]:
train_test_classes = ['Geometric Mean']
train_test_sets = ['Train Set', 'Test Set']

In [122]:
train_test_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut', 'No Cut','Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

train_test_row_names = pd.MultiIndex.from_product(
    [train_test_classes, train_test_sets],
    names=['Metric', 'Set'])

train_test_df = pd.DataFrame(
    index = train_test_row_names,
    dtype = float)

train_test_df = pd.DataFrame(
    columns = train_test_cols,
    index = train_test_row_names,
    dtype = float)

In [123]:
train_test_df['Instances'] = instances

train_test_df[('Time', 'Cut')] = t_cuts
train_test_df[('Time', 'No Cut')] = t_nocuts
train_test_df[('Time', 'RF')] = t_rf
train_test_df[('Time', 'ET')] = t_et
train_test_df[('Time', 'NN')] = t_nn

train_test_df[('Time', 'Perfect')] = t_perfect

train_test_df[('Node', 'Cut')] = n_cuts
train_test_df[('Node', 'No Cut')] = n_nocuts
train_test_df[('Node', 'RF')] = n_rf
train_test_df[('Node', 'ET')] = n_et
train_test_df[('Node', 'NN')] = n_nn

train_test_df[('Node', 'Perfect')] = n_perfect

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  180.289030  277.153358  154.992595  168.868150   
               Test Set   274.255724  347.103758  230.996876  271.396255   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  175.427147  164.576260  2610.230290   7071.275598   
               Test Set   271.396255  281.806623  5873.772645  12660.610956   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  2308.375949  2483.432574  2541.630897  2505.401045   
               Test Set   5074.993594  5856.937165  5856.937165  7024.195948   

Parameter                Instances  
Setting                             
Metric         Set                  
Geometric Mean Train Set       284  
               Test Set         71

In [124]:
best_time = []
best_nodes = []

zip_time = zip(t_rf, t_et, t_nn)
zip_time = list(zip_time)

zip_nodes = zip(n_rf, n_et, n_nn)
zip_nodes = list(zip_nodes)

for i in zip_time:
    best_time.append(min(i))

for i in zip_nodes:
    best_nodes.append(min(i))

In [125]:
imp_time = []
imp_nodes = []

for i, j in zip(best_time, t_cuts):
    imp = ((i - j)/j) * 100
    imp_time.append(imp)
    
for i, j in zip(best_nodes, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes.append(imp)

In [126]:
train_test_df[('Improvement (%)', 'Time')] = imp_time
train_test_df[('Improvement (%)', 'Nodes')] = imp_nodes

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  180.289030  277.153358  154.992595  168.868150   
               Test Set   274.255724  347.103758  230.996876  271.396255   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  175.427147  164.576260  2610.230290   7071.275598   
               Test Set   271.396255  281.806623  5873.772645  12660.610956   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  2308.375949  2483.432574  2541.630897  2505.401045   
               Test Set   5074.993594  5856.937165  5856.937165  7024.195948   

Parameter                Instances Improvement (%)            
Setting                                       Time     Nodes  
Metric         Set                                            
Geometric Mean Train Set       284       -8.715323 -4.857721  
               Test Set         71       -1.042629 -0.286621